In [3]:
import sqlite3
from IPython.display import Image

conn = sqlite3.connect('../database/database.db')

cursor = conn.cursor()


def query(input):
    
    cursor.execute(input)
    colnames = cursor.description
    for name in colnames:
        print(name[0])


    records = cursor.fetchall()

    for record in records:
        print(record)


In [2]:
query('''
      select *
      from OfficerDim
      ''')

NameError: name 'query' is not defined

In [2]:
query('''
      select * from vehicledim''')

Key
LPN
Make
Model
Year
OwnerDLN
OwnerName
OwnerGender
OwnerBirthYear
(31, 'IL2222', 'Honda', 'Civic', 2020, '111', 'Bill', 'Male', 1998)
(32, 'IL2222', 'Honda', 'Civic', 2020, '222', 'Suzy', 'Female', 1987)
(33, 'IL1111', 'Honda', 'CRV', 2017, '111', 'Bill', 'Male', 1998)


In [42]:
query('''
      select 
      
      case when id='a' then 'driving'
        else 'driving' end as Category,
      violation,
      fee 
      
      from drivingtickettype
    
      union
      
      select 
      
      case when id='a' then 'parking'
        else 'parking' end as Category,
      violation,
      fee
       
      from parkingtickettype

      ''')

Category
Violation
Fee
('driving', 'Red Light', 200)
('driving', 'Seat Belt', 100)
('parking', 'Hydrant', 150)
('parking', 'Meter Expired', 100)


In [44]:
query('select * from TicketTypeDim')

Key
Category
Violation
Fee
(1, 'driving', 'Red Light', 200)
(2, 'driving', 'Seat Belt', 100)
(3, 'parking', 'Hydrant', 150)
(4, 'parking', 'Meter Expired', 100)


In [71]:
query('''
      select
      -------------------------------------- case for days of week
      case cast (strftime('%w', date) as integer)
      when 0 then 'Sunday'
      when 1 then 'Monday'
      when 2 then 'Tuesday'
      when 3 then 'Wednesday'
      when 4 then 'Thursday'
      when 5 then 'Friday'
      else 'Saturday' end as DayOfWeek,
      strftime('%d', date) as DayOfMonth,
      strftime('%m', date) as Month,
      ------------------------------------- case for quarters
      case cast (strftime('m',date) as integer)
      when 1 | 2 | 3 then 'Q1'
      when 4 | 5 | 6 then 'Q2'
      when 7 | 8 | 9 then 'Q3'
      when 10 | 11 | 12 then 'Q4'
      else 'q4' end as Quarter,
      --------------------------------------
      
      strftime('%Y', date) as Year
      
      from DrivingTicket

      union all
      
      select
      -------------------------------------- case for days of week
      case cast (strftime('%w', date) as integer)
      when 0 then 'Sunday'
      when 1 then 'Monday'
      when 2 then 'Tuesday'
      when 3 then 'Wednesday'
      when 4 then 'Thursday'
      when 5 then 'Friday'
      else 'Saturday' end as DayOfWeek,
      strftime('%d', date) as DayOfMonth,
      strftime('%m', date) as Month,
      ------------------------------------- case for quarters
      case cast (strftime('m',date) as integer)
      when 1 | 2 | 3 then 'Q1'
      when 4 | 5 | 6 then 'Q2'
      when 7 | 8 | 9 then 'Q3'
      when 10 | 11 | 12 then 'Q4'
      else 'q4' end as Quarter,
      --------------------------------------
      
      strftime('%Y', date) as Year
      
      from ParkingTicket
      ''')

DayOfWeek
DayOfMonth
Month
Quarter
Year
('Wednesday', '01', '01', 'q4', '2020')
('Thursday', '02', '01', 'q4', '2020')
('Thursday', '02', '01', 'q4', '2020')
('Wednesday', '01', '01', 'q4', '2020')
('Wednesday', '01', '01', 'q4', '2020')
('Thursday', '02', '01', 'q4', '2020')


In [97]:
query('''
      select distinct date 
      from drivingticket
      
      union
      
      select distinct date
      from parkingticket
      ''')

Date
('2020-01-01',)
('2020-01-02',)


In [84]:
cursor.execute('delete from CalendarDim')
conn.commit()

In [101]:
query('select * from calendardim')

Key
Fulldate
DayOfWeek
DayOfMonth
Month
Quarter
Year
(33, '2020-01-01', 'Wednesday', 1, 1, 'q4', 2020)
(34, '2020-01-01', 'Thursday', 2, 1, 'q4', 2020)
(35, '2020-01-02', 'Wednesday', 1, 1, 'q4', 2020)
(36, '2020-01-02', 'Thursday', 2, 1, 'q4', 2020)


In [98]:
query('''
      with fulldate as (
            select distinct date 
            from drivingticket

            union

            select distinct date
            from parkingticket
      )
      
      select
      -------------------------------------- case for days of week
      case cast (strftime('%w', date) as integer)
      when 0 then 'Sunday'
      when 1 then 'Monday'
      when 2 then 'Tuesday'
      when 3 then 'Wednesday'
      when 4 then 'Thursday'
      when 5 then 'Friday'
      else 'Saturday' end as DayOfWeek,
      strftime('%d', date) as DayOfMonth,
      strftime('%m', date) as Month,
      ------------------------------------- case for quarters
      case cast (strftime('m',date) as integer)
      when 1 | 2 | 3 then 'Q1'
      when 4 | 5 | 6 then 'Q2'
      when 7 | 8 | 9 then 'Q3'
      when 10 | 11 | 12 then 'Q4'
      else 'q4' end as Quarter,
      --------------------------------------
      
      strftime('%Y', date) as Year
      
      from fulldate
      
      ''')

DayOfWeek
DayOfMonth
Month
Quarter
Year
('Wednesday', '01', '01', 'q4', '2020')
('Thursday', '02', '01', 'q4', '2020')


In [100]:
query('''
      with fulldate as (
            select distinct date 
            from drivingticket

            union

            select distinct date
            from parkingticket
      ),
      test as (
      select
      -------------------------------------- case for days of week
      case cast (strftime('%w', date) as integer)
      when 0 then 'Sunday'
      when 1 then 'Monday'
      when 2 then 'Tuesday'
      when 3 then 'Wednesday'
      when 4 then 'Thursday'
      when 5 then 'Friday'
      else 'Saturday' end as DayOfWeek,
      strftime('%d', date) as DayOfMonth,
      strftime('%m', date) as Month,
      ------------------------------------- case for quarters
      case cast (strftime('m',date) as integer)
      when 1 | 2 | 3 then 'Q1'
      when 4 | 5 | 6 then 'Q2'
      when 7 | 8 | 9 then 'Q3'
      when 10 | 11 | 12 then 'Q4'
      else 'q4' end as Quarter,
      --------------------------------------
      
      strftime('%Y', date) as Year
      
      from fulldate)
      select * from test
      
      ''')

DayOfWeek
DayOfMonth
Month
Quarter
Year
('Wednesday', '01', '01', 'q4', '2020')
('Thursday', '02', '01', 'q4', '2020')


In [18]:
query('''
      with payer as (
            select key PayerKey
            from payerdim
      ),
      
      vehicle as (
            select key VehicleKey
            from vehicledim
      ),
      
      calendar as (
            select key as CalendarKey
            from calendardim
      ),
      
      officer as (
            select key OfficerKey
            from officerdim
      ),
      
      ticket as (
            select key TicketTypeKey, fee as Amount
            from tickettypedim
      )
      
      
      select distinct
      Amount,
      CalendarKey,
      OfficerKey,
      PayerKey,
      VehicleKey,
      TicketTypeKey
      
      from 
      calendar, 
      ticket,
      payer, 
      vehicle, 
      officer
      
      ''')

Amount
CalendarKey
OfficerKey
PayerKey
VehicleKey
TicketTypeKey
(200, 45, 55, 7, 37, 45)
(200, 45, 56, 7, 37, 45)
(200, 45, 57, 7, 37, 45)
(200, 45, 55, 7, 38, 45)
(200, 45, 56, 7, 38, 45)
(200, 45, 57, 7, 38, 45)
(200, 45, 55, 7, 39, 45)
(200, 45, 56, 7, 39, 45)
(200, 45, 57, 7, 39, 45)
(200, 45, 55, 8, 37, 45)
(200, 45, 56, 8, 37, 45)
(200, 45, 57, 8, 37, 45)
(200, 45, 55, 8, 38, 45)
(200, 45, 56, 8, 38, 45)
(200, 45, 57, 8, 38, 45)
(200, 45, 55, 8, 39, 45)
(200, 45, 56, 8, 39, 45)
(200, 45, 57, 8, 39, 45)
(200, 45, 55, 9, 37, 45)
(200, 45, 56, 9, 37, 45)
(200, 45, 57, 9, 37, 45)
(200, 45, 55, 9, 38, 45)
(200, 45, 56, 9, 38, 45)
(200, 45, 57, 9, 38, 45)
(200, 45, 55, 9, 39, 45)
(200, 45, 56, 9, 39, 45)
(200, 45, 57, 9, 39, 45)
(100, 45, 55, 7, 37, 46)
(100, 45, 56, 7, 37, 46)
(100, 45, 57, 7, 37, 46)
(100, 45, 55, 7, 38, 46)
(100, 45, 56, 7, 38, 46)
(100, 45, 57, 7, 38, 46)
(100, 45, 55, 7, 39, 46)
(100, 45, 56, 7, 39, 46)
(100, 45, 57, 7, 39, 46)
(100, 45, 55, 8, 37, 46)
(100, 45, 5

In [19]:
query('''
      select key as CalendarKey
      from calendardim
      
      union
      
      select key as Officerkey
      from Officerdim
      
      union
      
      select key as PayerKey
      from payerdim
      
      union
      
      select key as VehicleKey
      from VehicleDim
      
      
      
      ''')

CalendarKey
(1,)
(2,)
(3,)
(31,)
(32,)
(33,)
(37,)
(38,)
(39,)
(40,)
(49,)
(50,)
(51,)


In [3]:
query('''
      select name
      from sqlite_master
      where type = "table";
      ''')

name
('sqlite_sequence',)
('PayerDim',)
('VehicleDim',)
('CalendarDim',)
('OfficerDim',)
('TicketTypeDim',)
('TicketRevenueFactTable',)
('Driver',)
('Vehicle',)
('Officer',)
('ParkingTicketType',)
('DrivingTicketType',)
('ParkingTicket',)
('DrivingTicket',)


In [16]:
query('''
      select *
      from ticketrevenuefacttable as fact
      join officerdim as od on od.key = fact.officerkey
      ''')

Id
Amount
CalendarKey
OfficerKey
PayerKey
VehicleKey
TicketTypeKey
Key
Id
Name
Rank


In [4]:
query('select * from ticketrevenuefacttable')

Id
Amount
CalendarKey
OfficerKey
PayerKey
VehicleKey
TicketTypeKey
(5, 200, 49, 58, 10, 40, 49)
(6, 100, 49, 58, 10, 40, 50)
(7, 150, 49, 58, 10, 40, 51)
(8, 100, 49, 58, 10, 40, 52)


In [11]:
query('''
      select amount, ownergender
      from ticketrevenuefacttable as fact
      join vehicledim as v on v.key = fact.vehiclekey
      group by ownergender
      ''')

Amount
OwnerGender
(200, 'Male')


In [31]:
cursor.execute('''
               delete from TicketRevenueFactTable
               ''')

In [20]:
cursor.execute('''
      with payer as (
            select key PayerKey
            from payerdim
      ),
      
      vehicle as (
            select key VehicleKey
            from vehicledim
      ),
      
      calendar as (
            select key as CalendarKey
            from calendardim
      ),
      
      officer as (
            select key OfficerKey
            from officerdim
      ),
      
      ticket as (
            select key TicketTypeKey, fee as Amount
            from tickettypedim
      )
      
      insert into TicketRevenueFactTable (Amount,CalendarKey,OfficerKey,PayerKey,VehicleKey,TicketTypeKey)
      select 
      Amount,
      CalendarKey,
      OfficerKey,
      PayerKey,
      VehicleKey,
      TicketTypeKey
      
      from 
      calendar, 
      ticket,
      payer, 
      vehicle, 
      officer
      
      ''')

In [7]:
query('''select sum(amount), gender 
      from ticketrevenuefacttable fact
      inner join payerdim payk on payk.key = fact.payerkey
      group by gender
      ''')

sum(amount)
Gender
(550, 'Male')
